# Threshold Joye-Libert Secure Aggregation

In [59]:
from ftsa.protocols.buildingblocks.JoyeLibert import TJLS
from ftsa.protocols.buildingblocks.VectorEncoding import VES
import random

t = 412
threshold = 7
nclients = 10
drops = 3
keysize = 2048 
inputsize = 16
dimension = 10

users = list(range(1, nclients+1))
alive = list(range(1, nclients-drops+1))
dropped = list(range(nclients-drops+1,nclients+1))


VE = VES(keysize // 2, nclients, inputsize, dimension)
TJL = TJLS(nclients, threshold, VE)

print("users: ", users)
print("alive: ", alive)
print("dropped: ", dropped)

users:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alive:  [1, 2, 3, 4, 5, 6, 7]
dropped:  [8, 9, 10]


### Generate JL keys (**Setup**)

In [60]:
# generate keys
pp, skey, ukey = TJL.Setup(keysize)

print(" - Public Parameters:", pp)
print(" - Aggregator Key:", skey)
print(" - User Keys:", ukey)

 - Public Parameters: <PublicParam (N=16075...03397, H(x)=0x7bbc)>
 - Aggregator Key: <ServerKey -0xbf6c1da>
 - User Keys: {0: <UserKey 0xac1de96f>, 1: <UserKey 0x17959e69>, 2: <UserKey 0xf71e793d>, 3: <UserKey 0x1892e4e2>, 4: <UserKey 0x14a07bab>, 5: <UserKey 0x1b84d79f>, 6: <UserKey 0x1e1df8a4>, 7: <UserKey 0x1087ecc4>, 8: <UserKey 0x176e5601>, 9: <UserKey 0x1f1b0b93>}


### Share the JL user key with all other users (**SKShare**)


In [61]:

# share secret key of each user
shares = {}
for u in users: 
    shares[u] = TJL.SKShare(ukey[u-1],threshold, users)

# distribute the shares
from collections import defaultdict
usersshares = defaultdict(dict)
for v in users:
    for u in users: 
        usersshares[u][v] = shares[v][u-1]

### Protect inputs of alive users with the users' keys (**Protect**)

In [62]:
# generate $nclients random vectors
L = []
for _ in range(nclients):
    l =[]
    for i in range(dimension):
        l.append(random.randint(0,1000))
    L.append(l)

# encrypt all vectors
E=[]
for u in alive:
    E.append(TJL.Protect(pp, ukey[u-1], t, L[u-1]))

E

[[<EncryptedNumber 16866...57161>,
  <EncryptedNumber 59212...60648>,
  <EncryptedNumber 15081...65203>,
  <EncryptedNumber 10905...26752>,
  <EncryptedNumber 56722...95975>,
  <EncryptedNumber 21518...06419>,
  <EncryptedNumber 12068...55545>,
  <EncryptedNumber 31834...15446>,
  <EncryptedNumber 93719...39654>,
  <EncryptedNumber 50089...87508>,
  <EncryptedNumber 15092...51511>,
  <EncryptedNumber 91780...35876>,
  <EncryptedNumber 21916...31533>,
  <EncryptedNumber 14922...89789>,
  <EncryptedNumber 25175...41974>,
  <EncryptedNumber 21130...02009>,
  <EncryptedNumber 13932...39907>,
  <EncryptedNumber 44120...47228>,
  <EncryptedNumber 18696...64089>,
  <EncryptedNumber 52405...21152>],
 [<EncryptedNumber 22987...05460>,
  <EncryptedNumber 24104...55810>,
  <EncryptedNumber 56595...78325>,
  <EncryptedNumber 63626...30252>,
  <EncryptedNumber 22688...47056>,
  <EncryptedNumber 20071...74207>,
  <EncryptedNumber 39318...43049>,
  <EncryptedNumber 22658...52536>,
  <EncryptedNumber 

### Protect the zero-value with the key shares of the dropped users  (**ShareProtect**)

In [63]:
if dropped != []:
    yzeroshare ={}
    for u in alive:
        dropped_users_shares = []
        for v in dropped:
            dropped_users_shares.append(usersshares[u][v])
        yzeroshare[u] = TJL.ShareProtect(pp, dropped_users_shares,t)


### Combine shares of the protected zero-value (**ShareCombine**)

In [64]:
if dropped != []:
    yzero = TJL.ShareCombine(pp, list(yzeroshare.values()), threshold)
else:
    yzero = None

### Aggregate all user protected inputs and the protected zero-value of dropped users (**Agg**)

In [65]:
S = TJL.Agg(pp, skey, t, E, yzero)


summ=L[0]
from operator import add
for l in L[1:nclients-drops]:
    summ = list(map(add, summ, l))

print(S)
# check if the result is correct
print("Verify: ", S == summ)

[4441, 3292, 4935, 4277, 2362, 3507, 3284, 3799, 4747, 3293, 2477, 3817, 3386, 4369, 4249, 3530, 3837, 2762, 3346, 2415, 3105, 3890, 3370, 3158, 4180, 1734, 4336, 4216, 2625, 4340, 4315, 1806, 3404, 2827, 2839, 4353, 3119, 1806, 4740, 2449, 4137, 3997, 2873, 3223, 3983, 4492, 4441, 3311, 2854, 3019, 5294, 3033, 3492, 3005, 2994, 4218, 4561, 3260, 3879, 3958, 4389, 4595, 3263, 2595, 3896, 3400, 4173, 3403, 2490, 4355, 3815, 4101, 2337, 4216, 3542, 2496, 4158, 3137, 3602, 4294, 3702, 2871, 3564, 2817, 3980, 4453, 3626, 4618, 3739, 2734, 2923, 4200, 3646, 3555, 2249, 3343, 3021, 5038, 4113, 2674, 3017, 3259, 4000, 3877, 2780, 4551, 3667, 4409, 3215, 4593, 1570, 2364, 2113, 2840, 3672, 3238, 4272, 3679, 3620, 3282, 2671, 3192, 3172, 3444, 3597, 3779, 3211, 4193, 3590, 2984, 3496, 3562, 2667, 4944, 3119, 4553, 2357, 2713, 3344, 3145, 3049, 2762, 3065, 2905, 2526, 3874, 4219, 3600, 2776, 4330, 3305, 4355, 2530, 3065, 3089, 3608, 3275, 4019, 3737, 4368, 2903, 2913, 4539, 3749, 4359, 3068, 228